# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d13/3f77d14c3d1fa1ac997d5414f38e394eeb9a734db495693c70daab1f.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Dubai, Dubai, United Arab Emirates**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib as mpl

%matplotlib notebook


def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

#leaflet_plot_stations(13,'3f77d14c3d1fa1ac997d5414f38e394eeb9a734db495693c70daab1f')

def paint_line_graph():
    df_source = pd.read_csv('data/C2A2_data/BinnedCsvs_d13/3f77d14c3d1fa1ac997d5414f38e394eeb9a734db495693c70daab1f.csv')
    df_source['Date'] = df_source['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    df_source['DateF'] = df_source['Date'].apply(lambda x: datetime.strftime(x, '%b-%d'))
    df_source['Data_Value'] = df_source['Data_Value'].apply(lambda x: int(x/10))
    
    df_2015 = df_source[df_source['Date'] >= '2015-01-01']
    df_bf2015 = df_source[df_source['Date'] < '2015-01-01']
        
    df_tmin_bf2015 = df_bf2015[df_bf2015['Element'] == 'TMIN'].pivot_table(values='Data_Value', aggfunc=np.min, index=['DateF'])
    df_tmin_bf2015 = df_tmin_bf2015.reset_index(drop=False)
    df_tmin_bf2015.rename(columns={'Data_Value':'min_temperature_records_bf2015'}, inplace=True)
    
    df_tmin_2015 = df_2015[df_2015['Element'] == 'TMIN'].pivot_table(values='Data_Value', aggfunc=np.min, index=['DateF'])
    df_tmin_2015 = df_tmin_2015.reset_index(drop=False)
    df_tmin_2015.rename(columns={'Data_Value':'min_temperature_records_2015'}, inplace=True)
    
    df_tmax_bf2015 = df_bf2015[df_bf2015['Element'] == 'TMAX'].pivot_table(values='Data_Value', aggfunc=np.max, index=['DateF'])
    df_tmax_bf2015 = df_tmax_bf2015.reset_index(drop=False)
    df_tmax_bf2015.rename(columns={'Data_Value':'max_temperature_records_bf2015'}, inplace=True)
    
    df_tmax_2015 = df_2015[df_2015['Element'] == 'TMAX'].pivot_table(values='Data_Value', aggfunc=np.max, index=['DateF'])
    df_tmax_2015 = df_tmax_2015.reset_index(drop=False)
    df_tmax_2015.rename(columns={'Data_Value':'max_temperature_records_2015'}, inplace=True)
    
    df_merged = pd.merge(df_tmin_bf2015, df_tmax_bf2015, right_on = 'DateF', left_on = 'DateF', how='inner')
    df_merged = pd.merge(df_merged, df_tmin_2015, right_on = 'DateF', left_on = 'DateF', how='inner')
    df_merged = pd.merge(df_merged, df_tmax_2015, right_on = 'DateF', left_on = 'DateF', how = 'inner')
    df_merged['max_temperature_records_2015'] = df_merged[['max_temperature_records_2015', 'max_temperature_records_bf2015' ]].apply(lambda x: np.nan if x['max_temperature_records_2015'] <= x['max_temperature_records_bf2015'] else x['max_temperature_records_2015'], axis=1)
    df_merged['min_temperature_records_2015'] = df_merged[['min_temperature_records_2015', 'min_temperature_records_bf2015']].apply(lambda x:  np.nan if x['min_temperature_records_2015'] >= x['min_temperature_records_bf2015'] else x['min_temperature_records_2015'] , axis=1)
    df_merged = df_merged[df_merged['DateF'] != 'Feb-29']
    df_merged['formattingDate'] = df_merged['DateF'].apply(lambda x: datetime.strptime(x,'%b-%d'))
    df_merged = df_merged.sort_values('formattingDate')
    df_merged.drop(['DateF'], axis=1, inplace=True)
    #df_merged = df_merged.set_index('formattedDate')
    
    #Obtain the lists for the line chart - before 2015
    max_temperature_bf2015 = df_merged['max_temperature_records_bf2015'].values
    min_temperature_bf2015 = df_merged['min_temperature_records_bf2015'].values
    observation_dates = df_merged['formattingDate'].values

    plt.figure(figsize=(8,6))
    plt.plot(observation_dates, max_temperature_bf2015, 'grey', observation_dates, min_temperature_bf2015, 'grey', alpha = 0.2)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.xlabel(' ')
    plt.ylabel('Temperature ($^\circ$C)', fontweight="bold")
    plt.title('Extreme weather in 2015', fontweight="bold")
    plt.gca().fill_between(observation_dates, 
                      min_temperature_bf2015, max_temperature_bf2015, 
                   facecolor='grey', 
                       alpha=0.1, label ='Temperature range 2005-2014')
  
    #Obtain the lists for the scatter plot - 2015
    max_temperature_2015 = df_merged['max_temperature_records_2015'].values
    min_temperature_2015 = df_merged['min_temperature_records_2015'].values
    plt.scatter(observation_dates, max_temperature_2015, s=10, c='red', label='Hotter in 2015')
    plt.scatter(observation_dates, min_temperature_2015, s=10, c='blue', label='Colder in 2015')
    plt.legend()
    plt.legend(loc=8, frameon=False, shadow=True, title = 'Legend', fontsize='small')
    plt.show()
    
paint_line_graph()



NameError: name 'df_tmin' is not defined